In [82]:
from utils import get_test_table
import numpy as np

In [83]:
X_train, X_test, y_train, y_test = get_test_table()
X_train = X_train.drop(["Unnamed: 0"], axis=1)
X_test=X_test.drop(["Unnamed: 0"], axis=1)

Encoding the date column

In [84]:
import pandas as pd

# Assuming 'X_train' and 'X_test' are pandas DataFrames and have a date column named 'date'
X_train['date'] = pd.to_datetime(X_train['date'])
X_test['date'] = pd.to_datetime(X_test['date'])

# Sort data by date
X_train = X_train.sort_values(by='date')
X_test = X_test.sort_values(by='date')

# Assign a unique number to each date
unique_dates = pd.concat([X_train['date'], X_test['date']]).unique()
date_to_number = {date: i for i, date in enumerate(sorted(unique_dates))}
X_train['date'] = X_train['date'].map(date_to_number).astype('float32')
X_test['date'] = X_test['date'].map(date_to_number).astype('float32')

# Convert the rest of your dataframe to float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Reshape for LSTM input
X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

Model

In [85]:
""" from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras import backend as K

def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=[rmse])

model.fit(X_train, y_train, epochs=5, batch_size=72, validation_data=(X_test, y_test), verbose=2, shuffle=False) """

" from keras.models import Sequential\nfrom keras.layers import LSTM, Dense\nfrom keras import backend as K\n\ndef rmse(y_true, y_pred):\n    return K.sqrt(K.mean(K.square(y_pred - y_true)))\n\nmodel = Sequential()\nmodel.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))\nmodel.add(Dense(1))\nmodel.compile(loss='mean_squared_error', optimizer='adam', metrics=[rmse])\n\nmodel.fit(X_train, y_train, epochs=5, batch_size=72, validation_data=(X_test, y_test), verbose=2, shuffle=False) "

In [89]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras import backend as K

param_grid = {
    'batch_size': [8, 16, 32, 64],
    'epochs': [3, 5, 7, 10]
}

def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

def create_compile_model(optimizer):
    model = Sequential()
    model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=[rmse])
    return model

best_score = None
best_params = None

for batch_size in param_grid['batch_size']:
    for epochs in param_grid['epochs']:

            model = create_compile_model('adam')

            model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=1, shuffle=False)

            score = model.evaluate(X_test, y_test, verbose=0)

            if best_score is None or score < best_score:
                best_score = score
                best_params = {'batch_size': batch_size, 'epochs': epochs}

print(f'Best score: {best_score}, with parameters: {best_params}')


Epoch 1/3
1336/1336 [==============================] - 10s 5ms/step - loss: 63.9408 - rmse: 7.1704 - val_loss: 60.0468 - val_rmse: 7.0312
Epoch 2/3
1336/1336 [==============================] - 6s 5ms/step - loss: 62.6471 - rmse: 7.1182 - val_loss: 60.0635 - val_rmse: 7.0359
Epoch 3/3
1336/1336 [==============================] - 6s 5ms/step - loss: 62.6138 - rmse: 7.1187 - val_loss: 60.0527 - val_rmse: 7.0329
Epoch 1/5
1336/1336 [==============================] - 10s 5ms/step - loss: 63.7631 - rmse: 7.1592 - val_loss: 60.0630 - val_rmse: 7.0357
Epoch 2/5
1336/1336 [==============================] - 6s 4ms/step - loss: 62.6558 - rmse: 7.1214 - val_loss: 60.0657 - val_rmse: 7.0365
Epoch 3/5
1336/1336 [==============================] - 6s 4ms/step - loss: 62.5368 - rmse: 7.1130 - val_loss: 60.0608 - val_rmse: 7.0351
Epoch 4/5
1336/1336 [==============================] - 6s 4ms/step - loss: 62.5777 - rmse: 7.1142 - val_loss: 60.0532 - val_rmse: 7.0330
Epoch 5/5
1336/1336 [==================